In [ ]:
import bibtexparser
from bibtexparser.bparser import BibTexParser 


def test_bibtexparser():
    """Test bibtexparser library
    https://bibtexparser.readthedocs.org/en/latest/tutorial.html
    """
    with open('wjpubs.bib') as bibtex_file:
        bib_db = bibtexparser.load(bibtex_file)
    print bib_db.entries



In [ ]:
#test_bibtexparser()

In [ ]:
import bibtexparser.customization as bcus


In [ ]:
# Let's define a function to customize our entries.
# It takes a record and return this record.
def customizations(record):
    """Use some functions delivered by the library

    :param record: a record
    :returns: -- customized record
    """
    record = bcus.type(record)
    record = bcus.author(record)
    record = bcus.editor(record)
    #record = bcus.journal(record)
    record = bcus.keyword(record)
    #record = bcus.link(record)
    record = bcus.page_double_hyphen(record)
    record = bcus.doi(record)
    return record


In [ ]:
import pprint 
fname = 'wjpubs.bib'
with open(fname) as bibtex_file:
    parser = BibTexParser()
    parser.customization = customizations
    bib_db = bibtexparser.load(bibtex_file, parser=parser)
    #print bib_db.entries
    pprint.pprint(bib_db.entries)

In [ ]:
def format_author(au_list, au_func=None):
    """au_func is a function taking in first name last name and return 
    an HTML formatted text.
    Format the author list into plain text"""
    
    fl_list = []
    for a in au_list:
        lf = [x.strip() for x in a.split(',')]
        fn = '%s %s'%(lf[1], lf[0])
        formatted = au_func(fn) if au_func is not None else fn
        fl_list.append(formatted)
    s = ', '.join(fl_list[:-1]) + ', and ' + fl_list[-1]
    return s

 
au_list = bib_db.entries[0]['author']
print format_author(au_list)

In [ ]:
from bibtexparser.bibdatabase import BibDatabase

def id2bibsrc(fname, ignore_key_list=None):
    src = {}
    with open(fname) as bibtext_file:
        raw_db = bibtexparser.load(bibtext_file)
        for en in raw_db.entries:
            tmp_db = BibDatabase()
                
            if ignore_key_list is not None:
                for k in ignore_key_list:
                    en.pop(k, None)
            
            tmp_db.entries = [en]
            en_src = bibtexparser.dumps(tmp_db)
            src[en['id']] = en_src
    return src

src = id2bibsrc(fname)
src['Jitkrittum2009']

In [ ]:
def clean_value(st):
    return st.replace('{', '').replace('}', '').replace('\&', '&')

def format_key(key, entry, map_func):
    form = map_func[key](entry[key]) if key in map_func else entry[key]
    form = clean_value(form)
    return form
                        
def format_bib_db(bib_db, map_func):
    """
    bib_db: a list of dictionaries from bibtexparser.load(..)
    map_func: a dict mapping from bibtex keys to functions to format the values
    associated with the keys"""
   
    html = ''
    for entry in bib_db:
        html += format_key('author', entry, map_func) + '.\n'
        html += format_key('title', entry, map_func) + '\n'
        #pprint.pprint(entry)
        if 'journal' in entry:
            html += format_key('journal', entry, map_func)
        elif 'booktitle' in entry:
            html += format_key('booktitle', entry, map_func)    
        html += ', '
        if 'year' in entry:
            html += format_key('year', entry, map_func)    
        html += '. '
        if 'link' in entry:
            html += 'URL: ' + format_key('link', entry, map_func)    
            
        html += '\n\n'
    return html
    

def author_func(author):
    a = author.strip()
    if a=='Wittawat Jitkrittum':
        return '<b>Wittawat Jitkrittum</b>'
    
    def link_author(text, href):
        return '<a href="%s">%s</a>'%(href, text)
    M = dict([
        ('Arthur Gretton', 'http://www.gatsby.ucl.ac.uk/~gretton/'),
        ('Nicolas Heess', 'http://homepages.inf.ed.ac.uk/s0677090/'),
        ('S. M. Ali Eslami', 'http://arkitus.com/'),
        ('Balaji Lakshminarayanan', 'http://www.gatsby.ucl.ac.uk/~balaji/'),
        ('Dino Sejdinovic', 'http://www.stats.ox.ac.uk/~sejdinov/'),
        (u'Zoltán Szabó', 'http://www.gatsby.ucl.ac.uk/~szabo/'),
        ('Mijung Park', 'http://www.gatsby.ucl.ac.uk/~mijung/'),
        ('Makoto Yamada', 'http://www.makotoyamada-ml.com/'),
        ('Leonid Sigal', 'http://cs.brown.edu/~ls/'),
        ('Eric P. Xing', 'http://www.cs.cmu.edu/~epxing/'),
        ('Masashi Sugiyama', 'http://www.ms.k.u-tokyo.ac.jp/index.html'),
        ('Hirotaka Hachiya', 'http://sugiyama-www.cs.titech.ac.jp/~hachiya/'),
        ('Gang Niu', 'http://sugiyama-www.cs.titech.ac.jp/~gang/'),
        ('Bo Dai', 'https://sites.google.com/site/daibohr/'), 

    ]);
    if a in M:
        return link_author(a, M[a])
    else:
        return a
    
map_func = {'author': format_author}
print format_bib_db(bib_db.entries, map_func)

In [ ]:

def format_html_bib_db(bib_db, map_func, map_src):
    """
    bib_db: a list of dictionaries from bibtexparser.load(..)
    map_func: a dict mapping from bibtex keys to functions to format the values
    associated with the keys. 
    map_src: a dict from bib key to bib source of the entry.
    
    Format the bib database into html. """
         
    html = '<div class="publications"> \n\n'
    for entry in bib_db:
        html += '<div class="paper"> \n'
        html += '<div class="paper_title">%s</div>\n'%( format_key('title', entry, map_func) )
        
        html += '<div class="author_list">%s</div>\n'%( format_key('author', entry, map_func) )
        #pprint.pprint(entry)
        place = ''
        if 'journal' in entry:
            place = format_key('journal', entry, map_func)
        elif 'booktitle' in entry:
            place = format_key('booktitle', entry, map_func)    
        html += '<span class="submit_to">%s</span>, '%place
        if 'year' in entry:
            html += format_key('year', entry, map_func)    
        html += '. \n'
        #if 'link' in entry:
        #    html += 'URL: ' + format_key('link', entry, map_func)  + '\n'
        html += '<div class="links">Link: \n'
        if 'wj_http' in entry:
            html += '<a class="wj_http" href="%s">http</a>\n'%( entry['wj_http'] )
        if 'wj_pdf' in entry:
            html += '<a class="wj_pdf" href="%s">pdf</a>\n'%( entry['wj_pdf'] )
        if 'wj_code' in entry:
            html += '<a class="wj_code" href="%s">code</a>\n'%( entry['wj_code'] )
        # bib link 
        html += '<a class="biblink">bib</a> \n'
        html += '</div> <!-- end links -->\n'
        # bib source. Can use jquery to make a sliding effect.
        html += '<pre class="bibsrc">%s</pre><!-- end bib source -->\n' % (map_src[entry['id']])
        
        # end paper
        html += '</div>'
        
        html += '\n\n'
    
    html += '</div> \n'
    return html
    
ignore_keys = ['wj_http', 'wj_pdf']
src = id2bibsrc(fname, ignore_keys)
map_func = {'author': lambda au_list: format_author(au_list, author_func)}
print format_html_bib_db(bib_db.entries, map_func, src)
